# Objectives:

1. Implement data preprocessing to summarize the features in the time series. The features hypothesized to be useful are as follows:
    * Band Spectrum Energy (eeg.get_power) (for EEG)
    * Fractal Dimension of Time Series (pyEEG)
    * Skewness and Kurtosis of the distributions (scipy.stats)
    * Regular stats (mean, median, max, min of time series)
    
2. Models to train:
    * Linear Chain Conditional Random Field with structured SVM solver with a 1-slack QP with L1 slack penalty (pystruct)
    * A simple SVM like Andreas (sklearn SVC)
    * A Hidden Markov model?
    
3. Cross Validate:
    * Build a proper function/system for leave-one-subject-out CV
    * Test models and ensembles on it


In [28]:
import importlib
import utils
importlib.reload(utils)
import utils
from utils import *

In [2]:
xtrain_eeg1, xtrain_eeg2, xtrain_emg, ytrain, xtest_eeg1, xtest_eeg2, xtest_emg = load_data()

Loading xtrain...
Shapes: (64800, 512) (64800, 512) (64800, 512)
Loading ytrain...
Shape: (64800, 1)
Loading xtest...
Shapes: (43200, 512) (43200, 512) (43200, 512)


# Model Implementation: Conditional 

In [ ]:
idx = 21600*2
xtrain_eeg1_ = xtrain_eeg1[0:idx]
xtrain_eeg2_ = xtrain_eeg2[0:idx]
xtrain_emg_ = xtrain_emg[0:idx]
eeg1_ = process_EEG(xtrain_eeg1_)
eeg2_ = process_EEG(xtrain_eeg2_)
emg_ = process_EMG(xtrain_emg_)
xtrain_ = np.concatenate((eeg1_, eeg2_, emg_), axis=1)
# ytrain_classes = ytrain[0:idx]['y']
ytrain_classes = ytrain.values[0:idx]
ytrain_ = ytrain.values[0:idx]

xtest_eeg1_ = xtrain_eeg1[idx:-1]
xtest_eeg2_ = xtrain_eeg2[idx:-1]
xtest_emg_ = xtrain_emg[idx:-1]
eeg1_ = process_EEG(xtest_eeg1_)
eeg2_ = process_EEG(xtest_eeg2_)
emg_ = process_EEG(xtest_emg_)
xtest_ = np.concatenate((eeg1_, eeg2_, emg_), axis=1)
ytest_ = ytrain.values[idx:-1]

# xtest_eeg1_ = xtrain_eeg1[idx:-1]
# xtest_eeg2_ = xtrain_eeg2[idx:-1]
# xtest_final = 

In [ ]:
# CRF

# xtrain_ = np.reshape(xtrain_, (xtrain_.shape[0], 1, xtrain_[0].shape[0])) # Reshape so that it works with CRF
xtrain_crf = np.reshape(xtrain_, (2, -1, xtrain_.shape[1])) # Reshape so that it works with CRF
ytrain_crf = np.reshape(ytrain_, (2, -1)) -1 # Reshape so that it works with CRF
# X_test_crf = X_test.reshape(2, -1, 49)
print(xtrain_crf.shape, ytrain_crf.shape)

print("Starting CRF...")
classes = np.array([1, 2, 3])
lmao = np.reshape(ytrain_classes, (-1,))
weights_crf = compute_class_weight("balanced", list(classes), list(lmao))
weights_crf[0] = weights_crf[0]+2.5
weights_crf[1] = weights_crf[1]+1.5
    
model = ChainCRF(class_weight=weights_crf)
ssvm = OneSlackSSVM(model=model, C=0.5, max_iter=2000)
    
ssvm.fit(xtrain_crf, ytrain_crf)   
   

# Test on the third guy
xtest_crf = np.reshape(xtest_, (1, -1, xtest_.shape[1]))
ytest_crf = np.reshape(ytest_, (1, -1)) -1
print(xtest_crf.shape, ytest_crf.shape)
y_pred_crf = ssvm.predict(xtest_crf)
y_pred_crf = np.asarray(y_pred_crf).reshape(-1) + 1  

print("BMAC:", sklearn.metrics.balanced_accuracy_score(ytest_, y_pred_crf))


# y_pred_filtered_crf = medfilt(y_pred_crf, FILTER_WINDOW)


# print(f"CRF unique predictions {np.unique(y_pred_filtered_crf)}")


# CRF Simply run model on CV

In [26]:
res = losocv_CRF(xtrain_eeg1, xtrain_eeg2, xtrain_emg, ytrain, C=0.1, weight_shift=1, fs=128)
print(res)

SIZE: (43200, 512)


AssertionError: Data must be of shape (nchan, n_samples).

# CRF Submission

In [11]:
pred = CRF_submit(xtrain_eeg1, xtrain_eeg2, xtrain_emg, ytrain, xtest_eeg1, xtest_eeg2, xtest_emg, C=0.1, weight_shift=0, fs=128)
make_submission('CRF0.1.csv', pred)


Gathering advanced statistics...


100%|██████████| 9/9 [00:00<00:00, 18.51it/s]
/Users/gabrielarpino/Documents/ETH_Semester_1/Advanced_Machine_Learning/sdaml/task4/gabriel/utils.py:398: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  (p, _, _, _) = numpy.linalg.lstsq(x, L)
/Users/gabrielarpino/Documents/ETH_Semester_1/Advanced_Machine_Learning/sdaml/task4/gabriel/utils.py:167: RuntimeWarning: divide by zero encountered in power
  np.sum((np.power(np.abs(signal),(-0.3)) > 20), axis=1),


Gathering advanced statistics...


100%|██████████| 9/9 [00:00<00:00, 18.34it/s]


Gathering advanced statistics...


...

100%|██████████| 9/9 [00:00<00:00, 28.09it/s]


Gathering advanced statistics...


100%|██████████| 9/9 [00:00<00:00, 27.77it/s]


Gathering advanced statistics...


100%|██████████| 9/9 [00:00<00:00, 28.11it/s]


# CRF Grid Search

In [ ]:
Cs = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0]
weight_shifts = [0]
total_res = []
for c, w in itertools.product(Cs, weight_shifts):
    res = losocv_CRF(xtrain_eeg1, xtrain_eeg2, xtrain_emg, ytrain, C=c, weight_shift=w, fs=128)
    total_res.append(res)
    print("C:", c, ", w:", w, ", BMAC (Mean, Std, full):", (np.mean(res), np.std(res), res))        
    print("********************************")

# Res December 14th, Cs and weight shift, EEG and EMG running process_EMG .
# C: 0.1 , w: 0 , BMAC (Mean, Std): (0.915638922375751, 0.05039793710589233)

# Dec 18
# C: 0.1 , w: 0 , BMAC (Mean, Std, full): (0.8105643639593416, 0.1823532025093181, [0.9057309406411344, 0.9705540211342516, 0.5554081301026389])
# C: 0.2 , w: 0 , BMAC (Mean, Std, full): (0.8320971763898081, 0.15734031410977364, [0.9146093715452001, 0.9698041054523155, 0.6118780521719084])

# Dec 18 grid search with EMG running process_EEG

/Users/gabrielarpino/Documents/ETH_Semester_1/Advanced_Machine_Learning/sdaml/task4/gabriel/utils.py:388: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  return p[0]
/Users/gabrielarpino/Documents/ETH_Semester_1/Advanced_Machine_Learning/sdaml/task4/gabriel/utils.py:144: RuntimeWarning: divide by zero encountered in power
  np.sum((np.power(np.abs(signal),(-0.3)) > 20), axis=1),
